In [1]:
%pylab inline

%load_ext autoreload
%autoreload 2

import pandas as pd
from py2neo import Graph
import itertools
pd.options.display.max_columns = 100

Populating the interactive namespace from numpy and matplotlib


In [2]:
graph = Graph('http://localhost:7474')

In [3]:
query = 'MATCH p = (source)--(destination) RETURN source, source.name, source.known_terrorist, avg(destination.known_terrorist)'
df = pd.DataFrame(graph.data(query))
df.head()

,avg(destination.known_terrorist),source,source.known_terrorist,source.name
0,0.250000,"{'name': 'Terrorist', 'person': '8', 'known_te...",1,Terrorist
1,0.000000,"{'name': 'Charlie', 'person': '3', 'known_terr...",0,Charlie
2,0.333333,"{'name': 'David', 'person': '4', 'known_terror...",0,David
3,0.250000,"{'name': 'Esther', 'person': '5', 'known_terro...",0,Esther
4,0.600000,"{'name': 'Fanny', 'person': '6', 'known_terror...",0,Fanny


query permutations
- directedness
- levels of depth
- type of connection

As the query already handles 3 queries at once, one would only need to loop over the data below:

In [4]:
df_all = None
r = itertools.product(*[
                         ['1','2','3', '1..2', '1..3'],
                         ['undirected', 'src->dst', 'src<-dst']])
for depth, direction in r:
    print((depth, direction))
    if 'src->dst' == direction:
        print('dsfsfdsf')
        dir_to_dest = '>'
        dir_to_src = ''
    elif 'src<-dst' == direction:
        dir_to_dest = ''
        dir_to_src = '<'
    else:
        dir_to_dest = ''
        dir_to_src = ''
    query = f"""
MATCH p = (source:Person){dir_to_src}-[:call|text*{depth}]-{dir_to_dest}(destination:Person)
RETURN 
  source.name as Vertex, 
  source.known_terrorist as known_terrorist,
  apoc.coll.avg(COLLECT(
    CASE WHEN ALL(r in relationships(p) where type(r)='call') THEN destination.known_terrorist ELSE NULL END
  )) as type_undir_1_call,
  apoc.coll.avg(COLLECT(
    CASE WHEN ALL(r in relationships(p) where type(r)='text') THEN destination.known_terrorist ELSE NULL END
  )) as type_undir_1_text,
  apoc.coll.avg(COLLECT(
    destination.known_terrorist
  )) as type_undir_1_any
"""
    #print(query)
    df = pd.DataFrame(graph.data(query))
    df['type'] = f'{depth}_{direction}'
    if df_all is None:
        df_all = df
    else:
        df_all = pd.concat([df_all, df])
    # display(df.head())
df_all.head()   

('1', 'undirected')
('1', 'src->dst')
dsfsfdsf
('1', 'src<-dst')
('2', 'undirected')
('2', 'src->dst')
dsfsfdsf
('2', 'src<-dst')
('3', 'undirected')
('3', 'src->dst')
dsfsfdsf
('3', 'src<-dst')
('1..2', 'undirected')
('1..2', 'src->dst')
dsfsfdsf
('1..2', 'src<-dst')
('1..3', 'undirected')
('1..3', 'src->dst')
dsfsfdsf
('1..3', 'src<-dst')


,Vertex,known_terrorist,type_undir_1_any,type_undir_1_call,type_undir_1_text,type
0,Gabby,0,1.000000,NaN,1.0,1_undirected
1,Esther,0,0.250000,0.5,0.0,1_undirected
2,Charlie,0,0.000000,NaN,0.0,1_undirected
3,David,0,0.333333,0.5,0.0,1_undirected
4,Terrorist,1,0.250000,0.5,0.0,1_undirected


reshape the `pandas.DataFrame`from long to wide

In [5]:
p = df_all.pivot('Vertex', 'type')
p

known_terrorist                                              \
type        1..2_src->dst 1..2_src<-dst 1..2_undirected 1..3_src->dst   
Vertex                                                                  
Alice                 1.0           1.0             1.0           1.0   
Bob                   0.0           0.0             0.0           0.0   
Charlie               0.0           0.0             0.0           0.0   
David                 0.0           0.0             0.0           0.0   
Esther                0.0           0.0             0.0           0.0   
Fanny                 0.0           0.0             0.0           0.0   
Gabby                 0.0           NaN             0.0           0.0   
Terrorist             1.0           1.0             1.0           1.0   

                                                                            \
type      1..3_src<-dst 1..3_undirected 1_src->dst 1_src<-dst 1_undirected   
Vertex                                                                       
Alice               1.0             1.0        1.0        1.0          1.0   
Bob                 0.0             0.0        0.0        0.0          0.0   
Charlie             0.0             0.0        0.0        0.0          0.0   
David               0.0             0.0        0.0        0.0          0.0   
Esther              0.0             0.0        0.0        0.0          0.0   
Fanny               0.0             0.0        0.0        0.0          0.0   
Gabby               NaN             0.0        0.0        NaN          0.0   
Terrorist           1.0             1.0        1.0        1.0          1.0   

                                                                    \
type      2_src->dst 2_src<-dst 2_undirected 3_src->dst 3_src<-dst   
Vertex                                                               
Alice            1.0        1.0          1.0        1.0        1.0   
Bob              0.0        0.0          0.0        NaN        0.0   
Charlie          0.0        0.0          0.0        NaN        0.0   
David            0.0        0.0          0.0        0.0        0.0   
Esther           0.0        0.0          0.0        0.0        0.0   
Fanny            0.0        0.0          0.0        0.0        0.0   
Gabby            0.0        NaN          0.0        0.0        NaN   
Terrorist        1.0        1.0          1.0        1.0        1.0   

                       type_undir_1_any                                \
type      3_undirected    1..2_src->dst 1..2_src<-dst 1..2_undirected   
Vertex                                                                  
Alice              1.0         0.142857      0.000000        0.066667   
Bob                0.0         0.000000      0.166667        0.181818   
Charlie            0.0         0.000000      0.333333        0.454545   
David              0.0         0.285714      0.333333        0.307692   
Esther             0.0         0.428571      0.200000        0.437500   
Fanny              0.0         0.333333      0.428571        0.368421   
Gabby              0.0         0.500000           NaN        0.400000   
Terrorist          1.0         0.500000      0.333333        0.350000   

                                                                             \
type      1..3_src->dst 1..3_src<-dst 1..3_undirected 1_src->dst 1_src<-dst   
Vertex                                                                        
Alice          0.375000      0.200000        0.311111   0.333333   0.000000   
Bob            0.000000      0.300000        0.344828   0.000000   0.500000   
Charlie        0.000000      0.285714        0.333333   0.000000   0.000000   
David          0.333333      0.200000        0.358974   0.500000   0.000000   
Esther         0.285714      0.285714        0.304348   0.000000   0.500000   
Fanny          0.363636      0.214286        0.295082   0.666667   0.500000   
Gabby          0.250000           NaN        0.133333   1.000